In [1]:
import glob
from itertools import product
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sourmash import signature as sig
from tqdm import tqdm

%matplotlib inline

## Info about the kmer hashes loaded

In [2]:
ksizes = 21, 27, 33, 51
ksize = ','.join(map(str, ksizes))
num_hashes = int(1e6)
dna = True
protein = True


In [3]:
output_prefix = 's3://olgabot-maca/facs/sourmash_compute_tissue_subset/'


In [4]:
annotations = pd.read_csv("/home/olga/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_facs.csv")
annotations['cell'] = annotations['cell'].str.replace('.', '-')
print(annotations.shape)
annotations.head()

(44949, 24)


/home/olga/miniconda3/envs/kmer-hashing/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Neurog3>0_raw,Neurog3>0_scaled,cell,cell_ontology_class,cell_ontology_id,cluster.ids,free_annotation,mouse.id,mouse.sex,plate.barcode,...,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subsetE,subsetE_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
0,NaN,NaN,A1-B000610-3_56_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_56_F,F,B000610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-7.182841,-10.895094
1,NaN,NaN,A1-B002764-3_38_F-1-1,bladder urothelial cell,CL:1001428,3,Luminal bladder epithelial cell,3_38_F,F,B002764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,4.199059,-11.050240
2,NaN,NaN,A1-B002771-3_39_F-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_39_F,F,B002771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-11.995435,-7.325534
3,NaN,NaN,A1-D041914-3_8_M-1-1,bladder cell,CL:1001319,5,Bladder mesenchymal cell,3_8_M,M,D041914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-6.820022,-14.174246
4,NaN,NaN,A1-D042253-3_9_M-1-1,bladder cell,CL:1001319,2,Bladder mesenchymal cell,3_9_M,M,D042253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bladder,-24.163538,-7.499349


In [5]:
! aws s3 ls $output_prefix | wc -l

1000


### Copy the files to ndnd

In [ ]:
! aws s3 sync $output_prefix /home/olga/scratch-pure/maca/facs/sourmash_compute_tissue_subset/

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-MAA100039-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-MAA100039-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-B002764-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-B002764-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A1-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/A1-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/A10-MAA000559-3_8_M-1-1.sig to ../../../s

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B10-MAA000910-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B10-MAA000910-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B1-MAA100138-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B1-MAA100138-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B10-MAA100039-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B10-MAA100039-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B11-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B11-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B12-MAA000577-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/B12-MAA000577-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/B11-MAA000801-3_11_M-1-1.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C19-MAA000776-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C19-MAA000776-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C19-MAA000779-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C19-MAA000779-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C2-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C2-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C2-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C2-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C2-MAA100140-3_57_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/C2-MAA100140-3_57_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/C20-MAA000577-3_8_M-1-1.sig to ../.

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D11-MAA100138-3_56_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D11-MAA100138-3_56_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D12-MAA100042-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D12-MAA100042-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D13-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D13-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D13-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D13-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D15-MAA000559-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/D15-MAA000559-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/D16-B000610-3_56_F-1-1.sig to ../

download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E21-MAA000577-3_8_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E21-MAA000577-3_8_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E22-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E22-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E22-MAA001868-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E22-MAA001868-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E3-MAA100040-3_11_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E3-MAA100040-3_11_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E3-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/E3-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/E4-MAA000801-3_11_M-1-1.sig to 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I15-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/I15-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I15-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/I15-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I15-MAA000752-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/I15-MAA000752-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I16-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/I16-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I16-D042253-3_9_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/I16-D042253-3_9_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/I17-B001717-3_38_F-1-1.sig to ../../.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J1-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/J1-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J11-MAA000487-3_10_M-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/J11-MAA000487-3_10_M-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J11-B000971-3_39_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/J11-B000971-3_39_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J11-B002764-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/J11-B002764-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J12-B001717-3_38_F-1-1.sig to ../../../scratch/maca/facs/sourmash_compute_tissue_subset/J12-B001717-3_38_F-1-1.sig
download: s3://olgabot-maca/facs/sourmash_compute_tissue_subset/J12-B001750-3_38_F-1-1.sig to ../../../

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
folder = '/home/olga/scratch-pure/maca/facs/sourmash_compute_tissue_subset/'

filenames = glob.glob(f'{folder}/*.sig')
downloaded_cells = [os.path.basename(f).split('.')[0] for f in filenames]
print(len(downloaded_cells))
downloaded_cells[:5]

In [ ]:
annotations_signatures = annotations.query('cell in @downloaded_cells')
annotations_signatures.head()

In [ ]:
sizes = annotations_signatures.groupby(['tissue', 'cell_ontology_class']).size()
sizes

In [ ]:
100 * sizes/200

In [ ]:
most_common_classes = sizes.groupby(level=0).nlargest(1)
most_common_classes

In [ ]:
most_common_classes.index.levels[-1]

In [ ]:
annotations_signatures_class_subset = annotations_signatures.query('cell_ontology_class in @most_common_classes.index.levels[-1]')
print(annotations_signatures_class_subset.shape)
annotations_signatures_class_subset.head()

In [ ]:
annotations_signatures.cell_ontology_class.mode()[0]

In [ ]:
annotations_signatures.sample()

In [ ]:
n = 10

def filter_by_top_classes(df, n, col='cell_ontology_class'):
    most_common_class = df[col].mode()[0]
    df = df.query(f'{col} == @most_common_class')
    return df.sample(n, random_state=0)
    

annotations_random_subset = annotations_signatures.groupby('tissue').apply(filter_by_top_classes, n=n)
print(annotations_random_subset.shape)
annotations_random_subset.head()

In [ ]:
annotations_random_subset.groupby(['tissue', 'cell_ontology_class']).size()

## Read signatures

In [ ]:
range(3)

In [ ]:

folder = '/home/olga/scratch/maca/facs/sourmash_compute_tissue_subset/'

filenames = folder + annotations_random_subset['cell'] + '.sig'
print(f'len(filenames): {len(filenames)}')

siglist_subset = []

for filename in tqdm(filenames):
    loaded = sig.load_signatures(filename)
    siglist_subset.extend(loaded)

In [ ]:
len(siglist_subset)

In [ ]:
siglist_subset[:5]

In [ ]:
%%time

import pickle

pickle_filename = f'{folder}/siglist_subset_npergroup{n}.pickle'

with open(pickle_filename, 'wb') as f:
    pickle.dump(siglist_subset, f)

In [ ]:
! ls -lha $folder/*pickle

In [4]:
range(3)

range(0, 3)

In [ ]:

folder = '/home/olga/scratch-pure/maca/facs/sourmash_compute_tissue_subset/'

filenames = glob.glob(f'{folder}/*.sig')


siglist = []

moltype = 'DNA'
ksize = 21


for filename in tqdm(filenames):
    loaded = sig.load_signatures(filename)
    siglist.extend(loaded)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
%%time

import pickle

pickle_filename = f'{folder}/siglist.pickle'

with open(pickle_filename, 'wb') as f:
    pickle.dump(siglist, f)

In [19]:
range(3)

range(0, 3)

In [20]:
ls -lha $pickle_filename

-rwxr-xr-x 1 olga olga 87G Nov  8 01:32 /home/olga/scratch/maca/facs/sourmash_compute_tissue_subset//siglist.pickle*
